In [ ]:
%%bash
git clone https://github.com/kptriescoding/farmer-app-datasets

Cloning into 'farmer-app-datasets'...


In [ ]:
import numpy as np
import pandas as pd
from scipy import spatial

In [ ]:
class RecommendationSystem:

  def __init__(self,data_path="/content/data.csv"):
    self.df= pd.read_csv(data_path)
    self.df.rename(columns={'FirstName LastName':"Name"},inplace=True)
    self.textFields= ["Name","Wheat Seller","Sugercane Seller","Rubber Seller","Maize Seller","Barley Seller"]

    self.productValues = {0:["Wheat Freqeuncy","Wheat Quantity"],1:["Sugercane Freqeuncy","Sugercane Quantity"],
                          2:["Rubber Freqeuncy","Rubber Quantity"],3:["Maize Freqeuncy","Maize Quantity"],
                          4:["Barley Freqeuncy","Barley Quantity"]}
    self.productCosines= pd.DataFrame(columns=["ReferenceIndex","ProductId","CosineSim"])

  
  def find_Cosine_Similarity(self, cust, numberOfRecommendations=10, numberOfPeople=10):

    cust_ind=self.df.index[self.df.Name==cust]
    custData=self.df[self.df["Name"]==cust]
    self.dfNumeric = self.df[self.df.columns.difference(self.textFields)]
    custData = custData[custData.columns.difference(self.textFields)]
    cosineInitials=np.full(shape=self.df.shape[0],fill_value=1.01)
    self.dfCosineSim=pd.DataFrame(cosineInitials,columns=["Cosine Similarity"])
    for ind in self.dfNumeric.index:
      if ind==cust_ind:
        continue;
      self.dfCosineSim["Cosine Similarity"].values[ind]=spatial.distance.cosine(custData.values[0],
                                                                                self.dfNumeric.values[ind])

    self.dfCosineSim.sort_values(by = "Cosine Similarity",inplace=True)
    self.topIndexes = [i for i in self.dfCosineSim.index[:numberOfPeople]]
    self.dfTopTen = self.dfNumeric.loc[self.topIndexes]
    # self.dfTopTen= self.dfCosineSim.head(numberOfPeople)
    self.newIndex=1
    
    for ind in range(0,numberOfPeople):
      for prod in self.productValues:
        try:
          prodCosine= spatial.distance.cosine(custData[self.productValues[prod]].values[0],
                                              self.dfTopTen[self.productValues[prod]].values[ind])
          self.productCosines.loc[len(self.productCosines)] = [ind,prod,prodCosine]
          self.newIndex+=1
        except Exception as e:
          print("Error: ",e)
    
    self.productCosines.sort_values(by = "CosineSim",inplace=True)

    print("Most Similar 5 suggestions are:\n",self.productCosines.head(int(numberOfRecommendations/2)),"\n")
    print("Most DiSimilar similar 5 suggestions are:\n",self.productCosines.tail(int(numberOfRecommendations/2)),"\n")

    self.dfSimilar = (self.productCosines.head(int(numberOfRecommendations/2))).ProductId
    self.dfDiSimilar = (self.productCosines.tail(int(numberOfRecommendations/2))).ProductId

    self.result = list()
    for i in range(0,int(numberOfRecommendations/2)):
      SimilarprodId= self.dfSimilar.values[i]
      self.result.append([SimilarprodId,(self.productValues[SimilarprodId][0]).split(" ")[0],"Similar"])
      DiSimilarprodId = self.dfDiSimilar.values[int(numberOfRecommendations/2)-i-1]
      self.result.append([DiSimilarprodId,(self.productValues[DiSimilarprodId][0]).split(" ")[0],"DiSimilar"])




    return self.result



In [ ]:
myrecommend = RecommendationSystem("/content/data.csv")

result = myrecommend.find_Cosine_Similarity("Wade Overson")
result

Most Similar 5 suggestions are:
     ReferenceIndex  ProductId     CosineSim
10             2.0        0.0  1.850635e-07
20             4.0        0.0  2.131967e-07
35             7.0        0.0  2.743465e-07
40             8.0        0.0  1.157502e-06
18             3.0        3.0  1.811293e-06 

Most DiSimilar similar 5 suggestions are:
     ReferenceIndex  ProductId  CosineSim
33             6.0        3.0        NaN
34             6.0        4.0        NaN
39             7.0        4.0        NaN
44             8.0        4.0        NaN
49             9.0        4.0        NaN 



/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


[[0.0, 'Wheat', 'Similar'],
 [4.0, 'Barley', 'DiSimilar'],
 [0.0, 'Wheat', 'Similar'],
 [4.0, 'Barley', 'DiSimilar'],
 [0.0, 'Wheat', 'Similar'],
 [4.0, 'Barley', 'DiSimilar'],
 [0.0, 'Wheat', 'Similar'],
 [4.0, 'Barley', 'DiSimilar'],
 [3.0, 'Maize', 'Similar'],
 [3.0, 'Maize', 'DiSimilar']]